In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds

In [2]:
import urllib3
urllib3.disable_warnings()

#tfds.disable_progress_bar()   # 이 주석을 풀면 데이터셋 다운로드과정의 프로그레스바가 나타나지 않습니다.

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
)

In [3]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    # image = tf.image.resize(image, [32, 32])
    return tf.cast(image, tf.float32) / 255., label

In [4]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [5]:
class VGG16(tf.keras.Model):
    def __init__(self):
        super(VGG16,self).__init__()
        self.conv1_1 = tf.keras.layers.Conv2D(64, kernel_size=3, strides=1, padding='same',activation='relu')
        self.conv1_2 = tf.keras.layers.Conv2D(64, kernel_size=3, strides=1, padding='same',activation='relu')
        self.maxpool1 = tf.keras.layers.MaxPool2D(pool_size=2)
        
        self.conv2_1 = tf.keras.layers.Conv2D(128, kernel_size=3, strides=1, padding='same',activation='relu')
        self.conv2_2 = tf.keras.layers.Conv2D(128, kernel_size=3, strides=1, padding='same',activation='relu')
        self.maxpool2 = tf.keras.layers.MaxPool2D(pool_size=2)
        
        self.conv3_1 = tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same',activation='relu')
        self.conv3_2 = tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same',activation='relu')
        self.conv3_3 = tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same',activation='relu')
        self.maxpool3 = tf.keras.layers.MaxPool2D(pool_size=2)
        
        self.conv4_1 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same',activation='relu')
        self.conv4_2 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same',activation='relu')
        self.conv4_3 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same',activation='relu')
        self.maxpool4 = tf.keras.layers.MaxPool2D(pool_size=2)
        
        self.conv5_1 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same',activation='relu')
        self.conv5_2 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same',activation='relu')
        self.conv5_3 = tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same',activation='relu')
        self.maxpool5 = tf.keras.layers.MaxPool2D(pool_size=2)

        self.gap = tf.keras.layers.GlobalAveragePooling2D()
        self.fc1 = tf.keras.layers.Dense(4096, activation='relu')
        self.fc2 = tf.keras.layers.Dense(4096, activation='relu')
        self.fc3 = tf.keras.layers.Dense(10, activation='softmax')        
        
    def call(self, input_tensor):
        x = self.conv1_1(input_tensor)
        x = self.conv1_2(x)
        x = self.maxpool1(x)
        
        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.maxpool2(x)
        
        x = self.conv3_1(x)
        x = self.conv3_2(x)
        x = self.conv3_3(x)
        x = self.maxpool3(x)
        
        x = self.conv4_1(x)
        x = self.conv4_2(x)
        x = self.conv4_3(x)
        x = self.maxpool4(x)
        
        x = self.conv5_1(x)
        x = self.conv5_2(x)
        x = self.conv5_3(x)
        x = self.maxpool5(x)
        
        x = self.gap(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [6]:
BATCH_SIZE = 256
EPOCH = 20

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)

In [7]:
vgg16 = VGG16()

vgg16.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_16 = vgg16.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1
)

Epoch 1/20
195/195 [==============================] - 13s 66ms/step - loss: 2.3025 - accuracy: 0.1010 - val_loss: 2.3025 - val_accuracy: 0.1317
Epoch 2/20
195/195 [==============================] - 12s 60ms/step - loss: 2.3025 - accuracy: 0.1112 - val_loss: 2.3024 - val_accuracy: 0.1566
Epoch 3/20
195/195 [==============================] - 12s 60ms/step - loss: 2.3024 - accuracy: 0.1187 - val_loss: 2.3024 - val_accuracy: 0.0968
Epoch 4/20
195/195 [==============================] - 12s 59ms/step - loss: 2.3023 - accuracy: 0.1112 - val_loss: 2.3023 - val_accuracy: 0.1024
Epoch 5/20
195/195 [==============================] - 12s 60ms/step - loss: 2.3023 - accuracy: 0.1046 - val_loss: 2.3023 - val_accuracy: 0.1010
Epoch 6/20
195/195 [==============================] - 11s 59ms/step - loss: 2.3022 - accuracy: 0.1085 - val_loss: 2.3022 - val_accuracy: 0.0960
Epoch 7/20
195/195 [==============================] - 11s 57ms/step - loss: 2.3022 - accuracy: 0.1125 - val_loss: 2.3020 - val_accuracy: